In [15]:
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Force TensorFlow to use CPU
tf.config.set_visible_devices([], 'GPU')  # Disables GPU
print("Using CPU for training.")

# Load the pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Update the output layer to match the number of classes
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(8, activation='softmax')(x)  # Update this to the correct number of classes

# Create the final model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
optimizer = Adam(learning_rate=1e-6)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Image data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1.0/255.0, horizontal_flip=True, rotation_range=20)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Define the directory path for train and validation sets
train_data_dir = r"F:\data_image\preprocessed_images"

# Paths for train and validation folders
train_dir = os.path.join(train_data_dir, "train")
val_dir = os.path.join(train_data_dir, "validation")

# Ensure the directories exist, create them if missing
if not os.path.exists(train_dir):
    print(f"Creating missing directory: {train_dir}")
    os.makedirs(train_dir)

if not os.path.exists(val_dir):
    print(f"Creating missing directory: {val_dir}")
    os.makedirs(val_dir)

# Check for class consistency
train_classes = set(os.listdir(train_dir))
val_classes = set(os.listdir(val_dir))

# Identify mismatched classes
missing_in_val = train_classes - val_classes
missing_in_train = val_classes - train_classes

# Fix folder structure by creating missing class directories
for class_name in missing_in_val:
    os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
for class_name in missing_in_train:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)

print("Folder structure aligned. Missing folders added where necessary.")

# Prepare the data generators
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    directory=val_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

# Verify class indices are consistent
if train_generator.class_indices != val_generator.class_indices:
    raise ValueError("Class indices in train and validation sets are inconsistent. Check folder structure.")

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Evaluate the model
val_generator.reset()
y_true = val_generator.classes
y_pred = model.predict(val_generator)
y_pred_classes = tf.argmax(y_pred, axis=1).numpy()

# Metrics
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print("Evaluation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes))

Using CPU for training.
Folder structure aligned. Missing folders added where necessary.
Found 4379 images belonging to 8 classes.
Found 1713 images belonging to 8 classes.


E:\Anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 3286s 12s/step - accuracy: 0.2554 - loss: 2.1362 - val_accuracy: 0.3339 - val_loss: 1.5078 - learning_rate: 1.0000e-06
Epoch 2/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 3270s 12s/step - accuracy: 0.3029 - loss: 1.6302 - val_accuracy: 0.3304 - val_loss: 1.4355 - learning_rate: 1.0000e-06
Epoch 3/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 3262s 12s/step - accuracy: 0.3256 - loss: 1.5605 - val_accuracy: 0.3368 - val_loss: 1.4109 - learning_rate: 1.0000e-06
Epoch 4/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 3271s 12s/step - accuracy: 0.3259 - loss: 1.5084 - val_accuracy: 0.3398 - val_loss: 1.3949 - learning_rate: 1.0000e-06
Epoch 5/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 3277s 12s/step - accuracy: 0.3272 - loss: 1.4901 - val_accuracy: 0.3438 - val_loss: 1.3851 - learning_rate: 1.0000e-06
Epoch 6/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 3271s 12s/step - accuracy: 0.3319 - loss: 1.4824 - val_accuracy: 0.3503 - val_loss: 1.3751 - learning_rate: 1.0000e-06
Epoch 7/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 3270s 

E:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: Undefine